In [14]:
%%pyspark

from pyspark.sql.types import FloatType

from pyspark.sql.functions import col, concat_ws
from pyspark.sql.functions import *

from pyspark.sql.types import TimestampType
from pyspark.sql.functions import unix_timestamp

### read the upload file into a dataframe
filewqual = "corp-signals-public-us-2020.csv"

### CHANGE THE @XXXXXX name to your storage workspace
thefile = "abfss://workspace@stgbvnews.dfs.core.windows.net/BitvoreDatasets/" + filewqual

df_raw = spark.read.options(header = 'True').csv(thefile)

df_raw.printSchema()

### Convert sentiment string to a float
### Convert strings representing date / times to spark timestamps and extract dates to new cols
df_conv = df_raw.withColumn("Sentiment", df_raw.Sentiment.cast('float'))\
                .withColumn("AvailableAt",unix_timestamp("AvailableAt", 'MM/dd/yyyy HH:mm').cast(TimestampType()))\
                .withColumn("dateAvailable", to_date(col("AvailableAt")) )\
                .withColumn("PublishedAt",unix_timestamp("PublishedAt", 'MM/dd/yyyy HH:mm').cast(TimestampType()))\
                .withColumn("datePub", to_date(col("PublishedAt")) )\
                .withColumn("LastModified",unix_timestamp("LastModified", 'MM/dd/yyyy HH:mm').cast(TimestampType()))\
                .withColumn("dateLastModified", to_date(col("LastModified")) )


df_conv.printSchema()

spark.sql("CREATE DATABASE IF NOT EXISTS bitvorenews")


### overwrite the  table
df_conv.write.mode("overwrite").saveAsTable("bitvorenews.signalspublic2020")




StatementMeta(bitvorespark, 13, 14, Finished, Available)

root
 |-- Key: string (nullable = true)
 |-- ArticleType: string (nullable = true)
 |-- AvailableAt: string (nullable = true)
 |-- PublishedAt: string (nullable = true)
 |-- CompanyId: string (nullable = true)
 |-- CompanyName: string (nullable = true)
 |-- Signal: string (nullable = true)
 |-- Sentiment: string (nullable = true)
 |-- SourceUrl: string (nullable = true)
 |-- SourceName: string (nullable = true)
 |-- PreviewImageUrl: string (nullable = true)
 |-- SimilarityId: string (nullable = true)
 |-- LastModified: string (nullable = true)

root
 |-- Key: string (nullable = true)
 |-- ArticleType: string (nullable = true)
 |-- AvailableAt: timestamp (nullable = true)
 |-- PublishedAt: timestamp (nullable = true)
 |-- CompanyId: string (nullable = true)
 |-- CompanyName: string (nullable = true)
 |-- Signal: string (nullable = true)
 |-- Sentiment: float (nullable = true)
 |-- SourceUrl: string (nullable = true)
 |-- SourceName: string (nullable = true)
 |-- PreviewImageUrl: string 

In [7]:
%%spark
val sql_write_df = spark.sql("SELECT * FROM bitvorenews.signalspublic2021")

sql_write_df.write.mode("overwrite").sqlanalytics("bitvoresql.dbo.signalspublic2021", Constants.INTERNAL)

StatementMeta(bitvorespark, 8, 5, Finished, Available)

StructuredStream-spark package version: 2.4.5-1.3.1
sql_write_df: org.apache.spark.sql.DataFrame = [Key: string, ArticleType: string ... 11 more fields]
StructuredStream-spark package version: 2.4.5-1.3.1
StructuredStream-spark package version: 2.4.5-1.3.1
StructuredStream-spark package version: 2.4.5-1.3.1
